In [1]:
import sqlite3
import numpy as np
# %load_ext cudf.pandas
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
# k-means法を使うためのインポート
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score


import sweetviz as sv

In [2]:
# SQLite データベースに接続
connect = sqlite3.connect("telecom.sqlite3")

# テーブル一覧を表示
cursor = connect.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# テーブル名を表示
for table in tables:
    print(table[0])

# 接続を閉じる
connect.close()


connect = sqlite3.connect("telecom.sqlite3")

record = pd.read_sql("select * from Record", con=connect)
client = pd.read_sql("select * from Client", con=connect)

uni_df = pd.merge(record, client, on="Customer_ID", how="inner")

uni_df = uni_df.dropna(subset=["change_mou"])

uni_df[uni_df["change_mou"].isna() == True]


analy_df = uni_df[["rev_Mean", "change_mou", "mou_Mean", "totmrc_Mean", "ovrmou_Mean", "ovrrev_Mean", "vceovr_Mean", "datovr_Mean", \
                  "months", "churn", "eqpdays"]].copy()

analy_df["MOUMEAN_PER_CHMOU"] = (analy_df["change_mou"]) / analy_df["mou_Mean"]
analy_df["revMean_PER_mouMean"] = ((analy_df["rev_Mean"]) / (analy_df["mou_Mean"]))

# ext_sources = ["avg6qty", "avg6mou"]

# for source in ext_sources:
#     analy_df[source].fillna(analy_df[source].mean(), inplace=True)

# analy_df = analy_df[0.1 < analy_df["revMean_PER_mouMean"]]

X = analy_df.drop(['churn'], axis=1)
y = analy_df['churn'] # 目的変数

X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size=0.2, random_state=0)

Record
Client


In [3]:

from lightgbm import LGBMClassifier

lgb_model = LGBMClassifier(random_state=0, learning_rate=0.02, num_leaves=26, max_depth=7, reg_alpha=0.1, reg_lambda=0.1)
lgb_model.fit(X_train, y_train)

lgb_train_pred = lgb_model.predict_proba(X_train)[:, 1]
lgb_valid_pred = lgb_model.predict_proba(X_valid)[:, 1]
print(f"Train Score: {roc_auc_score(y_train, lgb_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, lgb_valid_pred)}")

[LightGBM] [Info] Number of positive: 39193, number of negative: 40094
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005801 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2862
[LightGBM] [Info] Number of data points in the train set: 79287, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.494318 -> initscore=-0.022729
[LightGBM] [Info] Start training from score -0.022729
Train Score: 0.6683016987363903
Valid Score: 0.6599028630420798


In [4]:
lgb_model = LGBMClassifier(random_state=0, learning_rate=0.02, num_leaves=26, max_depth=7, reg_alpha=0.1, reg_lambda=0.1)
lgb_model.fit(X, y)

analy_df_pred = lgb_model.predict_proba(X)[:, 1]


lgb_train_pred_binary = (analy_df_pred >= 0.5).astype(int)

churned_data_train = analy_df[lgb_train_pred_binary == 1]

churned_data_train["churn"].mean()

[LightGBM] [Info] Number of positive: 48884, number of negative: 50225
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005778 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2862
[LightGBM] [Info] Number of data points in the train set: 99109, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493235 -> initscore=-0.027063
[LightGBM] [Info] Start training from score -0.027063


0.6118411522463423

In [5]:
churned_data_train["churn"].mean()

0.6118411522463423